# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hadibu,12.6500,54.0333,82.26,76,2,5.53,YE,1713650013
1,1,port-aux-francais,-49.3500,70.2167,46.85,94,97,32.32,TF,1713650014
2,2,puerto ayora,-0.7393,-90.3518,82.36,94,63,3.00,EC,1713650015
3,3,lihue,21.9789,-159.3672,79.84,68,20,12.66,US,1713650017
4,4,atafu village,-8.5421,-172.5159,85.06,76,2,12.44,TK,1713650018


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size='Humidity',
    color='City',
    cmap='Category20',
    tiles='OSM',
    frame_width=700,
    frame_height=480,
    hover_cols=['City'],
    legend='right'
)

# Display the map
map_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df[
    (city_data_df['Max Temp'] >= 65) &
    (city_data_df['Max Temp'] <= 75) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0)
]

# Drop any rows with null values, delete City_ID column, and reset the index
filtered_df = filtered_df.dropna().drop(columns=['City_ID']).reset_index(drop=True)

# Display sample data
filtered_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tsiombe,-25.3000,45.4833,68.74,95,0,2.62,MG,1713650251
1,chiredzi,-21.0500,31.6667,65.82,57,0,3.76,ZW,1713650290
2,gebeit,21.0667,36.3167,74.89,27,0,3.87,SD,1713650340
3,tamanrasset,22.7850,5.5228,69.71,6,0,2.30,DZ,1713650379
4,lashio,22.9333,97.7500,65.71,58,0,1.41,MM,1713650556
5,rakops,-21.0167,24.3333,74.39,37,0,3.65,BW,1713650837


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,tsiombe,MG,-25.3000,45.4833,95,
1,chiredzi,ZW,-21.0500,31.6667,57,
2,gebeit,SD,21.0667,36.3167,27,
3,tamanrasset,DZ,22.7850,5.5228,6,
4,lashio,MM,22.9333,97.7500,58,
5,rakops,BW,-21.0167,24.3333,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'limit': 10,
    'apiKey': f'{geoapify_key}'
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filter'] = f'circle:{lng},{lat},{radius}'
    params['bias'] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tsiombe - nearest hotel: No hotel found
chiredzi - nearest hotel: Nesbitt Arms Hotel
gebeit - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
lashio - nearest hotel: Golden Kaniri Hotel
rakops - nearest hotel: Rakops River Lodge


,City,Country,Lat,Lng,Humidity,Hotel Name
0,tsiombe,MG,-25.3000,45.4833,95,No hotel found
1,chiredzi,ZW,-21.0500,31.6667,57,Nesbitt Arms Hotel
2,gebeit,SD,21.0667,36.3167,27,No hotel found
3,tamanrasset,DZ,22.7850,5.5228,6,فندق الأمان
4,lashio,MM,22.9333,97.7500,58,Golden Kaniri Hotel
5,rakops,BW,-21.0167,24.3333,37,Rakops River Lodge


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size='Humidity',
    color='City',
    cmap='Category10',
    tiles='OSM',
    frame_width=700,
    frame_height=480,
    hover_cols=['City', 'Hotel Name', 'Country'],
    legend='right'
)

# Display the map
map_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)